In [4]:
from pyspark.sql import SparkSession

In [5]:

spark = SparkSession.builder \
    .appName('Ingest checkin table into bronze') \
    .master('spark://spark-master:7077') \
    .config("hive.metastore.uris", "thrift://hive-metastore:9083")\
    .config("spark.hadoop.fs.s3a.access.key", 'minio') \
    .config("spark.hadoop.fs.s3a.secret.key", 'minio123') \
    .config("spark.hadoop.fs.s3a.endpoint", 'minio:9000')\
    .config("spark.hadoop.fs.s3a.path.style.access", "true")\
    .config("spark.hadoop.fs.s3a.impl", "org.apache.hadoop.fs.s3a.S3AFileSystem") \
    .config("spark.hadoop.fs.s3a.connection.ssl.enabled", "false")\
    .config('spark.hadoop.fs.s3a.aws.credentials.provider', 'org.apache.hadoop.fs.s3a.SimpleAWSCredentialsProvider')\
    .config("spark.sql.extensions", "io.delta.sql.DeltaSparkSessionExtension") \
    .config("spark.sql.catalog.spark_catalog", "org.apache.spark.sql.delta.catalog.DeltaCatalog")\
    .config('spark.sql.warehouse.dir', f's3a://lakehouse/')\
    .enableHiveSupport()\
    .getOrCreate()

In [6]:
spark.sql('show schemas').show()

+---------+
|namespace|
+---------+
|  default|
|     gold|
|  platium|
|   silver|
+---------+



In [4]:
spark.sql("Select * from silver.cleaned_customer").limit(5).show()

+--------------------+--------------------+------------------------+--------------+--------------+
|         customer_id|  customer_unique_id|customer_zip_code_prefix| customer_city|customer_state|
+--------------------+--------------------+------------------------+--------------+--------------+
|0054556ea954a76ad...|3cc8e80baa86a7bef...|                   95680|        canela|            SP|
|009bd844996868ab5...|63df52c362d4b7183...|                   13076|      campinas|            SP|
|00abf30c1a93c7c8b...|d43e7cbf7354f1f46...|                    6226|        osasco|            SP|
|02625456293ab29f0...|570eb70ff97166b85...|                   31930|belo horizonte|            MG|
|028514f8be6e8c2ad...|be4d46b2c26abc028...|                   35200|       aimores|            MG|
+--------------------+--------------------+------------------------+--------------+--------------+



In [12]:
spark.sql("UPDATE silver.cleaned_customer set customer_state= 'SP' where customer_id = '0054556ea954a76ad6f9c4ba79d34a98'")

DataFrame[num_affected_rows: bigint]

In [13]:
spark.sql("SELECT count(*) FROM silver.cleaned_customer").show()

+--------+
|count(1)|
+--------+
|   99441|
+--------+



In [15]:
spark.sql("SELECT DISTINCT(customer_id) FROM silver.cleaned_customer").show(5,False)

+--------------------------------+
|customer_id                     |
+--------------------------------+
|36a1aa63bf2ebcd4911e026092700610|
|512f27d822abe6af95d86529e73724a6|
|384fbbcdcf45c174ca6407d4ade90112|
|4632eb5a8f175f6fe020520ae0c678f3|
|174cf4e5e95b5a49bac9cee9ef6cef70|
+--------------------------------+
only showing top 5 rows



## Delta Lake


### Transaction log

In [30]:
spark.sql("DESCRIBE HISTORY silver.cleaned_customer")

DataFrame[version: bigint, timestamp: timestamp, userId: string, userName: string, operation: string, operationParameters: map<string,string>, job: struct<jobId:string,jobName:string,runId:string,jobOwnerId:string,triggerType:string>, notebook: struct<notebookId:string>, clusterId: string, readVersion: bigint, isolationLevel: string, isBlindAppend: boolean, operationMetrics: map<string,string>, userMetadata: string, engineInfo: string]

In [31]:
spark.sql("DELETE FROM silver.cleaned_customer where customer_id = '0054556ea954a76ad6f9c4ba79d34a98'")

DataFrame[num_affected_rows: bigint]

### Schema Evolution

In [8]:
spark.sql("ALTER TABLE silver.cleaned_customer ADD COLUMN PHONE STRING")

DataFrame[]

In [10]:
spark.sql("SELECT * FROM silver.cleaned_customer").show(5,False)

+--------------------------------+--------------------------------+------------------------+--------------+--------------+-----+
|customer_id                     |customer_unique_id              |customer_zip_code_prefix|customer_city |customer_state|PHONE|
+--------------------------------+--------------------------------+------------------------+--------------+--------------+-----+
|009bd844996868ab5166ade7dd9ab471|63df52c362d4b7183bb6a463452a8183|13076                   |campinas      |SP            |null |
|00abf30c1a93c7c8b509cb80a22e4dd8|d43e7cbf7354f1f46a7a1b30701017b3|6226                    |osasco        |SP            |null |
|02625456293ab29f0b11a84835a8c0ab|570eb70ff97166b85ea96be3bfb65fef|31930                   |belo horizonte|MG            |null |
|028514f8be6e8c2adb9d0b4647ef3a39|be4d46b2c26abc0280f42d8325aa54a0|35200                   |aimores       |MG            |null |
|033225f6250f5ebf765eb90c41cd9c35|07b9a37d717b53a556e7c27669dfab57|83702                   |arauc

### Schema Enforcement

In [12]:
spark.sql("INSERT INTO silver.cleaned_customer VALUES ('119bd844996868abt166ade7dd9ab471','77df52c362c4b7183bb6a463452a8183',77665,'an nhon','an','0333333333')")

DataFrame[]

In [23]:
spark.sql("""
SELECT * FROM silver.cleaned_customer VERSION AS OF 3
""").show()

+--------------------+--------------------+------------------------+-------------------+--------------+-----+
|         customer_id|  customer_unique_id|customer_zip_code_prefix|      customer_city|customer_state|PHONE|
+--------------------+--------------------+------------------------+-------------------+--------------+-----+
|009bd844996868ab5...|63df52c362d4b7183...|                   13076|           campinas|            SP| null|
|00abf30c1a93c7c8b...|d43e7cbf7354f1f46...|                    6226|             osasco|            SP| null|
|02625456293ab29f0...|570eb70ff97166b85...|                   31930|     belo horizonte|            MG| null|
|028514f8be6e8c2ad...|be4d46b2c26abc028...|                   35200|            aimores|            MG| null|
|033225f6250f5ebf7...|07b9a37d717b53a55...|                   83702|          araucaria|            PR| null|
|03514f9ba6258fadc...|3a4f5d794447ef719...|                   75115|           anapolis|            GO| null|
|04bf1a1d1

In [29]:
spark.sql("""
SELECT * FROM silver.cleaned_customer TIMESTAMP AS OF '2024-07-01 13:50:00'
""").show()

+--------------------+--------------------+------------------------+-------------------+--------------+
|         customer_id|  customer_unique_id|customer_zip_code_prefix|      customer_city|customer_state|
+--------------------+--------------------+------------------------+-------------------+--------------+
|0054556ea954a76ad...|3cc8e80baa86a7bef...|                   95680|             canela|            RS|
|009bd844996868ab5...|63df52c362d4b7183...|                   13076|           campinas|            SP|
|00abf30c1a93c7c8b...|d43e7cbf7354f1f46...|                    6226|             osasco|            SP|
|02625456293ab29f0...|570eb70ff97166b85...|                   31930|     belo horizonte|            MG|
|028514f8be6e8c2ad...|be4d46b2c26abc028...|                   35200|            aimores|            MG|
|033225f6250f5ebf7...|07b9a37d717b53a55...|                   83702|          araucaria|            PR|
|03514f9ba6258fadc...|3a4f5d794447ef719...|                   75

### cache table

In [7]:
spark.sql("""
CACHE TABLE silver.cleaned_customer
""")

DataFrame[]

In [11]:
spark.sql("""
SELECT * FROM silver.cleaned_customer
""").show()

+--------------------+--------------------+------------------------+-------------------+--------------+-----+
|         customer_id|  customer_unique_id|customer_zip_code_prefix|      customer_city|customer_state|PHONE|
+--------------------+--------------------+------------------------+-------------------+--------------+-----+
|009bd844996868ab5...|63df52c362d4b7183...|                   13076|           campinas|            SP| null|
|00abf30c1a93c7c8b...|d43e7cbf7354f1f46...|                    6226|             osasco|            SP| null|
|02625456293ab29f0...|570eb70ff97166b85...|                   31930|     belo horizonte|            MG| null|
|028514f8be6e8c2ad...|be4d46b2c26abc028...|                   35200|            aimores|            MG| null|
|033225f6250f5ebf7...|07b9a37d717b53a55...|                   83702|          araucaria|            PR| null|
|03514f9ba6258fadc...|3a4f5d794447ef719...|                   75115|           anapolis|            GO| null|
|04bf1a1d1